# アンサンブル学習

#### 小さなデータセットの用意
>以前も利用した回帰のデータセットを用意します。
>
>House Prices: Advanced Regression Techniques
>
>この中のtrain.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使います。
>
>train.csvを学習用（train）8割、検証用（val）2割に分割してください。
>
#### scikit-learn
>単一のモデルはスクラッチ実装ではなく、scikit-learnなどのライブラリの使用を推奨します。

## 【問題1】ブレンディングのスクラッチ実装  
>ブレンディング をスクラッチ実装し、単一モデルより精度があがる例を 最低3つ 示してください。精度があがるとは、検証用データに対する平均二乗誤差（MSE）が小さくなることを指します。



### ブレンディングとは
>ブレンディングとは、N個の多様なモデルを独立して学習させ、推定結果を重み付けした上で足し合わせる方法です。最も単純には平均をとります。多様なモデルとは、以下のような条件を変化させることで作り出すものです。
>
>- 手法（例：線形回帰、SVM、決定木、ニューラルネットワークなど）
>- ハイパーパラメータ（例：SVMのカーネルの種類、重みの初期値など）
>- 入力データの前処理の仕方（例：標準化、対数変換、PCAなど）
>
>重要なのはそれぞれのモデルが大きく異なることです。
>
>回帰問題でのブレンディングは非常に単純であるため、scikit-learnには用意されていません。
>
>《補足》
>
>分類問題の場合は、多数決を行います。回帰問題に比べると複雑なため、scikit-learnにはVotingClassifierが用意されています。

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# データセットcsvをpandasに読み込む

csv_path = "./Kaggle_data/train.csv" # ファイル名（パス）を指定する
df_data = pd.read_csv(csv_path)

# 条件に従って抜き出し
df_X = df_data[['GrLivArea', 'YearBuilt']]
df_y = df_data['SalePrice']

df = pd.concat([df_X, df_y], axis=1)

display(df)

,GrLivArea,YearBuilt,SalePrice
0,1710,2003,208500
1,1262,1976,181500
2,1786,2001,223500
3,1717,1915,140000
4,2198,2000,250000
...,...,...,...
1455,1647,1999,175000
1456,2073,1978,210000
1457,2340,1941,266500
1458,1078,1950,142125


In [3]:
# 特徴量（説明変数）をX、正解（目的変数）をyというndarrayに格納

X = np.array(df[['GrLivArea','YearBuilt']])
y = np.array(df['SalePrice'])
print(X)
print(y)

[[1710 2003]
 [1262 1976]
 [1786 2001]
 ...
 [2340 1941]
 [1078 1950]
 [1256 1965]]
[208500 181500 223500 ... 266500 142125 147500]


In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(1168, 2)
(1168,)
(292, 2)
(292,)


In [5]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [6]:
#from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor

#from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LinearRegression

#from sklearn.svm import SVC
from sklearn.svm import SVR

#from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_squared_error

In [7]:
# 精度の一覧表用の空のリストを用意
verification_result = []


#最近傍法
verification_neigh = ['k-nearest neighbors']

# 近傍点を５に設定して学習
neigh_baseline = KNeighborsRegressor(n_neighbors=5)# インスタンス化
neigh_baseline.fit(X_train_scaled, y_train)# 学習
y_pred_n = neigh_baseline.predict(X_test_scaled)# 予測

# 平均二乗誤差（Mean Squared Error, MSE）    
verification_neigh.append(mean_squared_error(y_test,  y_pred_n))


# 線形回帰
verification_linear = ['LinearRegression']

linear_baseline = LinearRegression()
linear_baseline.fit(X_train_scaled, y_train)
y_pred_li = linear_baseline.predict(X_test_scaled)

# 平均二乗誤差（Mean Squared Error, MSE）    
verification_linear.append(mean_squared_error(y_test,  y_pred_li))



# SVM
verification_svm = ['SVM']

svm_baseline = SVR()
svm_baseline.fit(X_train_scaled, y_train)
y_pred_s = svm_baseline.predict(X_test_scaled)

# 平均二乗誤差（Mean Squared Error, MSE）
verification_svm.append(mean_squared_error(y_test, y_pred_s))



# 決定木
verification_tree = ['DecisionTree']

tree_baseline = DecisionTreeRegressor()
tree_baseline.fit(X_train_scaled, y_train)
y_pred_t = tree_baseline.predict(X_test_scaled)

# 平均二乗誤差（Mean Squared Error, MSE）
verification_tree.append(mean_squared_error(y_test, y_pred_t))



# 表を作成するために計算結果を２次元配列にする
verification_result = [
            verification_neigh,
            verification_linear, 
            verification_svm, 
            verification_tree, 
        ]


# 行と列のインデックスようのリストを用意
data_columns=['Model', '平均二乗誤差（MSE）']

# pandas のデータフレームにする
df_verification = pd.DataFrame(data=verification_result, columns=data_columns)

print("今回のベースライン")
display(df_verification)


今回のベースライン


,Model,平均二乗誤差（MSE）
0,k-nearest neighbors,2.138338e+09
1,LinearRegression,2.942067e+09
2,SVM,7.221625e+09
3,DecisionTree,3.157409e+09


最近傍法、線形回帰、決定木、SVMの順に精度が高い。

In [8]:
class ScratchEnsembleLearning():
    """
    アンサンブル学習

    Parameters
    ----------
    verbose : bool
      学習過程を出力する場合はTrue
      
    Attributes
    ----------
    """
    def __init__(self, verbose=False):
        # ハイパーパラメータを属性として記録
        self.verbose = verbose
        
        
    def fit(self, X, y, model_instance, weight):
        """
        学習する
        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            訓練データの特徴量
        y : 次の形のndarray, shape (n_samples, )
            訓練データの正解値
        """
        INSTANCE_0 = model_instance[0]
        INSTANCE_1 = model_instance[1]
        INSTANCE_2 = model_instance[2]

        INSTANCE_0.fit(X, y)
        INSTANCE_1.fit(X, y)
        INSTANCE_2.fit(X, y)
        
        self.instance_0 = INSTANCE_0
        self.instance_1 = INSTANCE_1
        self.instance_2 = INSTANCE_2
        
        
    def predict(self, X_test):
        """
        推定する
        """
        pred_0 = self.instance_0.predict(X_test)
        pred_1 = self.instance_1.predict(X_test)
        pred_2 = self.instance_2.predict(X_test)
        
        pred = (pred_0*weight[0] + pred_1*weight[1] + pred_2*weight[2])#/3
        
        self.pred = pred
        self.pred_0 = pred_0
        self.pred_1 = pred_1
        self.pred_2 = pred_2
        
        return pred
    
    
    def MSE(self, y_test, y_pred):
        """
        平均二乗誤差（Mean Squared Error, MSE）
        """
        MSE = mean_squared_error(y_test, y_pred)
        
        # 精度の一覧表用の空のリストを用意
        verification_result = []
        MSE_0 = mean_squared_error(y_test, self.pred_0)
        MSE_1 = mean_squared_error(y_test, self.pred_1)
        MSE_2 = mean_squared_error(y_test, self.pred_2)
        
        # 表を作成するために計算結果を２次元配列にする
        verification_MSE_0 = ['INSTANCE_0', MSE_0]
        verification_MSE_1 = ['INSTANCE_1', MSE_1]
        verification_MSE_2 = ['INSTANCE_2', MSE_2]
        verification_MSE = ['EnsembleLearning', MSE]
        verification_result = [
                    verification_MSE_0, 
                    verification_MSE_1, 
                    verification_MSE_2, 
                    verification_MSE
                ]
        # pandas のデータフレームにする
        df_verification = pd.DataFrame(data=verification_result, columns=data_columns)
        display(df_verification)
        
        return MSE
        

In [9]:
# インスタンス化
neigh = KNeighborsRegressor(n_neighbors=5)
linear = LinearRegression()
svm = SVR()
tree = DecisionTreeRegressor()

#### パターン１  
ベースラインが良かったもの３つを均等な比率で組み合わせる。

In [10]:
# 引数
model_instance = [neigh, linear, tree]
weight = [0.34, 0.33, 0.33]

# インスタンス化
ensemble_learning_1 = ScratchEnsembleLearning()
ensemble_learning_1.fit(X_train_scaled, y_train, model_instance, weight)
y_pred = ensemble_learning_1.predict(X_test_scaled)
ensemble_learning_1.MSE(y_test, y_pred)

,Model,平均二乗誤差（MSE）
0,INSTANCE_0,2.138338e+09
1,INSTANCE_1,2.942067e+09
2,INSTANCE_2,3.194443e+09
3,EnsembleLearning,2.292421e+09


2292421141.83418

精度が低い決定木に引っ張られて全体的に精度が下がった。

#### パターン２  
最近傍法がさらに強く出るように比率を変更。また、決定木は誤差が大きいのであまり効かないよう重みを下げた。

In [60]:
# 引数
model_instance = [neigh, linear, tree]
weight = [0.7, 0.2, 0.1]

# インスタンス化
ensemble_learning_2 = ScratchEnsembleLearning()
ensemble_learning_2.fit(X_train_scaled, y_train, model_instance, weight)
y_pred = ensemble_learning_2.predict(X_test_scaled)
ensemble_learning_2.MSE(y_test, y_pred)

,Model,平均二乗誤差（MSE）
0,INSTANCE_0,2.138338e+09
1,INSTANCE_1,2.942067e+09
2,INSTANCE_2,3.121344e+09
3,EnsembleLearning,2.139811e+09


2139811438.859748

精度は上がったが、まだ最近傍法のベースラインよりは誤差が大きい状況。

#### パターン3  
最近傍法が精度が高いのでさらに比率を上げて実行。

In [38]:
# 引数
model_instance = [neigh, linear, tree]
weight = [0.8, 0.1, 0.1]

# インスタンス化
ensemble_learning_4 = ScratchEnsembleLearning()
ensemble_learning_4.fit(X_train_scaled, y_train, model_instance, weight)
y_pred = ensemble_learning_4.predict(X_test_scaled)
ensemble_learning_4.MSE(y_test, y_pred)

,Model,平均二乗誤差（MSE）
0,INSTANCE_0,2.138338e+09
1,INSTANCE_1,2.942067e+09
2,INSTANCE_2,3.316931e+09
3,EnsembleLearning,2.141752e+09


2141751678.838851

かなりベースラインの精度に近づいてきたことを確認。

#### パターン４ 
これまで使用していた中で最も精度が低い決定木をSVMに変更。

In [39]:
# 引数
model_instance = [neigh, linear, svm]
weight = [0.8, 0.1, 0.1]

# インスタンス化
ensemble_learning_3 = ScratchEnsembleLearning()
ensemble_learning_3.fit(X_train_scaled, y_train, model_instance, weight)
y_pred = ensemble_learning_3.predict(X_test_scaled)
ensemble_learning_3.MSE(y_test, y_pred)

,Model,平均二乗誤差（MSE）
0,INSTANCE_0,2.138338e+09
1,INSTANCE_1,2.942067e+09
2,INSTANCE_2,7.221625e+09
3,EnsembleLearning,2.190372e+09


2190372216.170055

やや精度が下がった。

#### パターン５
線形回帰をSVMに変更。

In [43]:
# 引数
model_instance = [neigh, svm, tree]
weight = [0.8, 0.1, 0.1]

# インスタンス化
ensemble_learning_3 = ScratchEnsembleLearning()
ensemble_learning_3.fit(X_train_scaled, y_train, model_instance, weight)
y_pred = ensemble_learning_3.predict(X_test_scaled)
ensemble_learning_3.MSE(y_test, y_pred)

,Model,平均二乗誤差（MSE）
0,INSTANCE_0,2.138338e+09
1,INSTANCE_1,7.221625e+09
2,INSTANCE_2,3.338991e+09
3,EnsembleLearning,2.149598e+09


2149597785.5427303

ベースラインよりも精度が高いモデルを作ることができた。

### 結論  
最近傍法, SVM, 決定木を0.8, 0.1, 0.1の重みづけで出力すると一番誤差が少なく、  
ベースラインよりも精度が上がることが分かった。  
個別で精度が高いモデルの比率が高い方がいい結果が出やすいことがわかった。  
一方で、個別で精度が低いモデルと入れ替えてみると組み合わせがうまくいったのか、  
全体の精度を引き上げることがあるということが分かった。

## 【問題2】バギングのスクラッチ実装  
>バギング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。



## バギングとは  
>バギングは入力データの選び方を多様化する方法です。学習データから重複を許した上でランダムに抜き出すことで、N種類のサブセット（ **ブートストラップサンプル** ）を作り出します。それらによってモデルをN個学習し、推定結果の平均をとります。ブレンディングと異なり、それぞれの重み付けを変えることはありません。
>
>sklearn.model_selection.train_test_split — scikit-learn 0.21.3 documentation
>
>scikit-learnのtrain_test_splitを、shuffleパラメータをTrueにして使うことで、ランダムにデータを分割することができます。これによりブートストラップサンプルが手に入ります。
>
>推定結果の平均をとる部分はブースティングと同様の実装になります。

In [14]:
import copy

class ScratchBagging():
    """
    バギング

    Parameters
    ----------
    verbose : bool
      学習過程を出力する場合はTrue
      
    Attributes
    ----------
    """
    def __init__(self, verbose=False):
        # ハイパーパラメータを属性として記録
        self.verbose = verbose
        
        
    def fit(self, X, y, model_instance, test_size=0.2):
        """
        学習する
        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            訓練データの特徴量
        y : 次の形のndarray, shape (n_samples, )
            訓練データの正解値
        """
        X_train_0, _ , y_train_0, _ = train_test_split(X, y, test_size=test_size, shuffle=True)
        X_train_1, _ , y_train_1, _ = train_test_split(X, y, test_size=test_size, shuffle=True)  
        X_train_2, _ , y_train_2, _ = train_test_split(X, y, test_size=test_size, shuffle=True)
        
        model_0 = model_instance[0]
        model_1 = model_instance[1]
        model_2 = model_instance[2]
        
        model_0.fit(X_train_0, y_train_0)
        model_1.fit(X_train_1, y_train_1)
        model_2.fit(X_train_2, y_train_2)
        
        self.instance_0 = model_0
        self.instance_1 = model_1
        self.instance_2 = model_2
        
        
    def predict(self, X):
        """
        推定する
        """
        pred_tmp_0 = self.instance_0.predict(X)
        pred_tmp_1 = self.instance_1.predict(X)
        pred_tmp_2 = self.instance_2.predict(X)
        
        pred = (pred_tmp_0+pred_tmp_1+pred_tmp_2)/3
        
        self.pred_0 = pred_tmp_0
        self.pred_1 = pred_tmp_1
        self.pred_2 = pred_tmp_2
        
        return pred
    
    
    def MSE(self, y_test, y_pred):
        """
        平均二乗誤差（Mean Squared Error, MSE）
        """
        MSE = mean_squared_error(y_test, y_pred)
        
        # 精度の一覧表用の空のリストを用意
        verification_result = []
        MSE_0 = mean_squared_error(y_test, self.pred_0)
        MSE_1 = mean_squared_error(y_test, self.pred_1)
        MSE_2 = mean_squared_error(y_test, self.pred_2)
        
        # 表を作成するために計算結果を２次元配列にする
        verification_MSE_0 = ['INSTANCE_0', MSE_0]
        verification_MSE_1 = ['INSTANCE_1', MSE_1]
        verification_MSE_2 = ['INSTANCE_3', MSE_2]
        verification_MSE = ['Bagging', MSE]
        verification_result = [
                    verification_MSE_0, 
                    verification_MSE_1, 
                    verification_MSE_2, 
                    verification_MSE
                ]
        # pandas のデータフレームにする
        df_verification = pd.DataFrame(data=verification_result, columns=data_columns)
        display(df_verification)
        
        return MSE
        

#### パターン1
ベースラインの結果が良かった最近傍法で実施。

In [49]:
# 引数
svm_bagging_0 = KNeighborsRegressor(n_neighbors=5)
svm_bagging_1 = KNeighborsRegressor(n_neighbors=5)
svm_bagging_2 = KNeighborsRegressor(n_neighbors=5)

model_instance = [svm_bagging_0, svm_bagging_1, svm_bagging_2]

# インスタンス化
bagging_1 = ScratchBagging()
bagging_1.fit(X_train_scaled, y_train, model_instance, test_size=0.2)
y_pred_b = bagging_1.predict(X_test_scaled)
bagging_1.MSE(y_test, y_pred_b)

,Model,平均二乗誤差（MSE）
0,INSTANCE_0,2.110598e+09
1,INSTANCE_1,2.015035e+09
2,INSTANCE_3,2.048022e+09
3,Bagging,2.011157e+09


2011157381.2615824

#### パターン２
ベースライン２番目に良かった線形回帰で実施。

In [50]:
# 引数
linear_bagging_0 = LinearRegression()
linear_bagging_1 = LinearRegression()
linear_bagging_2 = LinearRegression()

model_instance = [linear_bagging_0, linear_bagging_1, linear_bagging_2]

# インスタンス化
bagging_1 = ScratchBagging()
bagging_1.fit(X_train_scaled, y_train, model_instance, test_size=0.2)
y_pred_b = bagging_1.predict(X_test_scaled)
bagging_1.MSE(y_test, y_pred_b)

,Model,平均二乗誤差（MSE）
0,INSTANCE_0,2.988697e+09
1,INSTANCE_1,2.958038e+09
2,INSTANCE_3,2.903096e+09
3,Bagging,2.945249e+09


2945249416.5680766

#### パターン３
３番目に精度が高かった決定木で実施。

In [51]:
# 引数
tree_bagging_0 = DecisionTreeRegressor()
tree_bagging_1 = DecisionTreeRegressor()
tree_bagging_2 = DecisionTreeRegressor()

model_instance = [tree_bagging_0, tree_bagging_1, tree_bagging_2]

# インスタンス化
bagging_1 = ScratchBagging()
bagging_1.fit(X_train_scaled, y_train, model_instance, test_size=0.2)
y_pred_b = bagging_1.predict(X_test_scaled)
bagging_1.MSE(y_test, y_pred_b)

,Model,平均二乗誤差（MSE）
0,INSTANCE_0,3.526428e+09
1,INSTANCE_1,3.063766e+09
2,INSTANCE_3,3.692474e+09
3,Bagging,2.701435e+09


2701435297.8975983

#### パターン４
ベースラインがよくなかったSVMで実施。

In [52]:
# 引数
neigh_bagging_0 = SVR()
neigh_bagging_1 = SVR()
neigh_bagging_2 = SVR()

model_instance = [neigh_bagging_0, neigh_bagging_1, neigh_bagging_2]

# インスタンス化
bagging_1 = ScratchBagging()
bagging_1.fit(X_train_scaled, y_train, model_instance, test_size=0.2)
y_pred_b = bagging_1.predict(X_test_scaled)
bagging_1.MSE(y_test, y_pred_b)

,Model,平均二乗誤差（MSE）
0,INSTANCE_0,7.183578e+09
1,INSTANCE_1,7.354046e+09
2,INSTANCE_3,7.186242e+09
3,Bagging,7.237039e+09


7237039467.082136

### 結論  
最近傍法, 決定木はバギングによって精度が向上した。

## 【問題3】スタッキングのスクラッチ実装  
>スタッキング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。



In [19]:
from sklearn.model_selection import KFold

In [57]:
class ScratchStacking():
    """
    スタッキング

    Parameters
    ----------
    verbose : bool
      学習過程を出力する場合はTrue
      
    Attributes
    ----------
    self.pred_list
    　　
    self.instance_list
    　　
    self.instance_l2
      
    """
    def __init__(self, verbose=False):
        # ハイパーパラメータを属性として記録
        self.verbose = verbose
        
        
    def fit(self, X, y, INSTANCE, INSTANCE_L2):
        """
        学習する
        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            訓練データの特徴量
        y : 次の形のndarray, shape (n_samples, )
            訓練データの正解値
        """
        ############ ステージ０　############
        # 推定保管用
        pred_list = []
        
        # モデル数分ループ
        for i in range(3):
            # データを４分割
            kf = KFold(n_splits=4)
            # データ分割を実施
            kf.get_n_splits(X)
            
            # 推定保管用
            pred_tmp = []
            
            # ループ回数のカウント用
            j = 0
            #for j, (train_index, test_index) in enumerate(zip(kf.split(X))):
            for train_index, test_index in kf.split(X):
                X_train, X_test = X[train_index], X[test_index]
                y_train, y_test = y[train_index], y[test_index]
                
                # 学習
                INSTANCE[i][j].fit(X_train, y_train)
                # 推定
                y_pred = INSTANCE[i][j].predict(X_test)
                pred_tmp.extend(y_pred)
                
                j += 1
            
            # OOF
            pred_list.append(pred_tmp)

        # OOF の保管
        self.pred_list = np.array(pred_list).T
        # インスタンスの保管
        self.instance_list = np.array(INSTANCE)
        
        ############ ステージN　############
        # pred を 学習用に分割
        X_train_l2, X_test_l2, y_train_l2, y_test_l2 = train_test_split(self.pred_list, y, test_size=0.2, random_state=0)

        # 学習
        INSTANCE_L2.fit(X_train_l2, y_train_l2)
        # インスタンスの保管
        self.instance_l2 = INSTANCE_L2
        
        
    def predict(self, X):
        """
        推定する
        """
        ############ ステージ０　############
        # 推定保管用
        pred_list = []
        for i in range(3):
            # 推定保管用
            pred_tmp = []
            for j in range(4):
                # 推定
                y_pred = self.instance_list[i, j].predict(X)
                pred_tmp.append(y_pred)
            # 同一モデルないで平均を取る
            pred_tmp = np.mean(np.array(pred_tmp).T, axis=1)
            pred_list.append(pred_tmp)
            
        # 推定データの作成
        pred_list = np.array(pred_list).T
        
        ############ ステージN　############
        y_pred_l2 = self.instance_l2.predict(pred_list)
        print(y_pred_l2)
        
        return y_pred_l2
    
    
    def MSE(self, y_test, y_pred):
        """
        平均二乗誤差（Mean Squared Error, MSE）
        """
        MSE = mean_squared_error(y_test, y_pred)
        
        return MSE
        

#### パターン1
ベースラインとして単独で実施したときに最も精度の良かったSVCのみで構成して実施。

In [58]:
# 引数

# L１のインスタンス
neigh_stacking_a0 = KNeighborsRegressor()
neigh_stacking_a1 = KNeighborsRegressor()
neigh_stacking_a2 = KNeighborsRegressor()
neigh_stacking_a3 = KNeighborsRegressor()
instance_a = [neigh_stacking_a0, neigh_stacking_a1, neigh_stacking_a2, neigh_stacking_a3]


neigh_stacking_b0 = KNeighborsRegressor()
neigh_stacking_b1 = KNeighborsRegressor()
neigh_stacking_b2 = KNeighborsRegressor()
neigh_stacking_b3 = KNeighborsRegressor()
instance_b = [neigh_stacking_b0, neigh_stacking_b1, neigh_stacking_b2, neigh_stacking_b3]


neigh_stacking_c0 = KNeighborsRegressor()
neigh_stacking_c1 = KNeighborsRegressor()
neigh_stacking_c2 = KNeighborsRegressor()
neigh_stacking_c3 = KNeighborsRegressor()
instance_c = [neigh_stacking_c0, neigh_stacking_c1, neigh_stacking_c2, neigh_stacking_c3]

model_instance = [instance_a, instance_b, instance_c]

# L2のインスタンス
l2__instance = KNeighborsRegressor()



# インスタンス化
stacking_1 = ScratchStacking()
stacking_1.fit(X_train_scaled, y_train, model_instance, l2__instance)

# 推定
y_pred_b = stacking_1.predict(X_test_scaled)
stacking_1.MSE(y_test, y_pred_b)

[227577.  135860.  130300.  182580.  122305.   83540.  227577.  125280.
 416557.4 151800.  208780.  168080.  229386.4 140080.  137080.  153800.
 224033.  142100.  151780.  140760.  152200.  139100.  119770.  167920.
 198069.  124500.  248795.6 117700.  269824.2 134900.  197609.  210800.
 131780.  259383.4 358433.6 168149.6 211400.  118400.  285259.6 290704.6
 227880.  161700.  218477.  306701.8 399943.4 166800.  136380.  141560.
 188320.  111076.6 493854.  141290.  151800.   97551.2 213600.  122200.
 123770.  266840.  139300.  112760.  152140.  154180.  153067.4 182740.
 203480.  166700.  112700.  290704.6 109900.  222200.  171440.  103158.6
 120600.  166269.  112700.  302001.8 126280.  112760.  323180.  164720.
 133210.  138140.  124920.  142100.  284359.8 170520.  101280.  208200.
 178680.  159300.  215839.  188580.  211392.6 258640.  190220.  134500.
 200520.  146800.  121200.  154400.  235338.  229265.6 134580.  182060.
  84480.  322800.  174860.  101600.  211560.  123600.  101460.

2201189723.286301

単独で実施した時が2138338000のため、やや精度が下がった。  
単独のモデルを使うより、いろいろなモデルを組み合わせる方が良いと思われる。

#### パターン２
ステージ０をSVM、最近傍法、決定木で構成。最終ステージは線形回帰で構成。

In [54]:
# 引数

# L１のインスタンス
svm_stacking_0 = SVR()
svm_stacking_1 = SVR()
svm_stacking_2 = SVR()
svm_stacking_3 = SVR()
instance_a = [svm_stacking_0, svm_stacking_1, svm_stacking_2, svm_stacking_3]

neigh_stacking_0 = KNeighborsRegressor(n_neighbors=5)
neigh_stacking_1 = KNeighborsRegressor(n_neighbors=5)
neigh_stacking_2 = KNeighborsRegressor(n_neighbors=5)
neigh_stacking_3 = KNeighborsRegressor(n_neighbors=5)
instance_b = [neigh_stacking_0, neigh_stacking_1, neigh_stacking_2, neigh_stacking_3]

tree_stacking_0 = DecisionTreeRegressor()
tree_stacking_1 = DecisionTreeRegressor()
tree_stacking_2 = DecisionTreeRegressor()
tree_stacking_3 = DecisionTreeRegressor()
instance_c = [tree_stacking_0, tree_stacking_1, tree_stacking_2, tree_stacking_3]

model_instance = [instance_a, instance_b, instance_c]

# L2のインスタンス
l2__instance = LinearRegression()



# インスタンス化
stacking_1 = ScratchStacking()
stacking_1.fit(X_train_scaled, y_train, model_instance, l2__instance)

# 推定
y_pred_b = stacking_1.predict(X_test_scaled)
stacking_1.MSE(y_test, y_pred_b)

[207859.16150009 142898.01735589 123820.15145362 194757.53938354
 120767.50754581  84551.38196558 207305.83909884 124219.36888441
 443864.3615429  155258.03141519 204980.77232381 170142.21331652
 238830.57669727 123009.86199102 126736.75256127 147135.21054727
 224716.31885018 139826.90128446 153071.57518187 160798.17658257
 156840.83944005 144226.49705424 114486.77524109 198257.79972328
 183377.35770557 125517.90072559 235400.02904751 112045.15688217
 305630.59835506 129155.1605065  183825.35746603 209127.86110221
 128467.94926608 259178.75297014 347278.01050069 203349.48018841
 213293.9130798  119277.55222869 311658.21251957 308972.36092774
 219540.6793343  154324.44959388 207236.72116203 281059.24283766
 366596.6611251  154902.54819515 110589.28548879 126900.8782707
 181885.58889151 119158.09550088 393258.65163679 139346.51213015
 154969.02686515  89541.1471766  196189.96435609 117274.34988881
 130003.49074388 251246.25892811 144430.69402358 102775.4405392
 137790.57796308 138799.340

2068602443.268628

単に４つのモデルを組み合わせただけで精度が向上し、すべてのベースラインを上回った。

#### パターン３  
最終ステージは精度の高い最近傍法で構成。

In [59]:
# 引数

# L１のインスタンス
svm_stacking_0 = SVR()
svm_stacking_1 = SVR()
svm_stacking_2 = SVR()
svm_stacking_3 = SVR()
instance_a = [svm_stacking_0, svm_stacking_1, svm_stacking_2, svm_stacking_3]

tree_stacking_0 = DecisionTreeRegressor()
tree_stacking_1 = DecisionTreeRegressor()
tree_stacking_2 = DecisionTreeRegressor()
tree_stacking_3 = DecisionTreeRegressor()
instance_b = [tree_stacking_0, tree_stacking_1, tree_stacking_2, tree_stacking_3]

linear_stacking_0 = LinearRegression()
linear_stacking_1 = LinearRegression()
linear_stacking_2 = LinearRegression()
linear_stacking_3 = LinearRegression()
instance_c = [linear_stacking_0, linear_stacking_1, linear_stacking_2, linear_stacking_3]

model_instance = [instance_a, instance_b, instance_c]

# L2のインスタンス
l2__instance = KNeighborsRegressor(n_neighbors=5)



# インスタンス化
stacking_1 = ScratchStacking()
stacking_1.fit(X_train_scaled, y_train, model_instance, l2__instance)

# 推定
y_pred_b = stacking_1.predict(X_test_scaled)
stacking_1.MSE(y_test, y_pred_b)

[308837.2 161800.  114950.  251272.6 135768.8  94955.2 181400.  127200.
 410354.8 166930.  189990.  181510.  263658.  124380.  119480.  139800.
 261514.4 157640.  127490.  151640.  126060.  148860.  120140.  191617.
 204600.  123960.  210648.6 111950.  270600.  127350.  163906.6 169600.
 137590.  308910.6 292604.8 185700.  200700.  127850.  286452.2 326820.
 259418.  158050.  197659.  204960.  470348.8 176800.  106160.  136800.
 181160.  104340.  300689.4 147900.  160780.  107800.  213800.  124380.
 124370.  190860.  131900.  113500.  133100.  128430.  158040.  149900.
 202448.6 137280.  120901.6 301190.  106180.  222530.  208800.  131700.
 112680.8 163780.  116156.6 263200.  132460.  112180.  309380.  146160.
 141680.  127100.  101480.8 157640.  325843.8 172178.   93880.  174968.
 191160.  132580.  176400.  187879.  210316.  223208.  180279.  165000.
 172413.  139760.  126500.  143400.  210660.  266450.6 142600.  155000.
  72842.2 376367.4 192520.  106000.  210720.  111280.8  66322.2 

2898919417.7445207

逆に精度が下がったことを確認。

#### パターン４  
最終ステージを決定木に変更して実施。

In [56]:
# 引数

# L１のインスタンス
svm_stacking_0 = SVR()
svm_stacking_1 = SVR()
svm_stacking_2 = SVR()
svm_stacking_3 = SVR()
instance_a = [svm_stacking_0, svm_stacking_1, svm_stacking_2, svm_stacking_3]

neigh_stacking_0 = KNeighborsRegressor(n_neighbors=5)
neigh_stacking_1 = KNeighborsRegressor(n_neighbors=5)
neigh_stacking_2 = KNeighborsRegressor(n_neighbors=5)
neigh_stacking_3 = KNeighborsRegressor(n_neighbors=5)
instance_b = [neigh_stacking_0, neigh_stacking_1, neigh_stacking_2, neigh_stacking_3]

linear_stacking_0 = LinearRegression()
linear_stacking_1 = LinearRegression()
linear_stacking_2 = LinearRegression()
linear_stacking_3 = LinearRegression()
instance_c = [linear_stacking_0, linear_stacking_1, linear_stacking_2, linear_stacking_3]

model_instance = [instance_a, instance_b, instance_c]

# L2のインスタンス
l2__instance = DecisionTreeRegressor()



# インスタンス化
stacking_1 = ScratchStacking()
stacking_1.fit(X_train_scaled, y_train, model_instance, l2__instance)

# 推定
y_pred_b = stacking_1.predict(X_test_scaled)
stacking_1.MSE(y_test, y_pred_b)

[350000. 146150. 119500. 210000. 132500. 109500. 185000. 119500. 184750.
 180000. 254000. 187000. 211000. 157500. 124000. 170000. 215000. 150750.
 165500. 115000. 143000. 140000. 139000. 177000. 183000. 124000. 179500.
  68400. 255000. 144000. 159895. 277000. 144000. 394617. 465000. 254000.
 277000. 129000. 239686. 260000. 277000. 178000. 254000. 318061. 466500.
 143900. 110000. 127500. 173500.  62383. 380000. 143000. 180000.  80000.
 185000. 127000. 114504. 235000. 142000. 163500. 129500. 128000. 150000.
 150000. 325000.  97000. 153900. 297000.  68400. 210000. 185000. 128000.
 153900. 173500. 153900. 293077. 108000. 163500. 340000. 143000. 143750.
 108000. 124000. 150750. 302000. 157000. 116900. 177000. 160000. 143000.
 277000. 157000. 185000. 199900. 185000. 152000. 185000. 157000. 127000.
 113000. 235000. 254900. 152000. 149500.  73000. 275500. 164700. 115000.
 185000. 114504. 100000. 143000. 173000. 128000. 173500. 160000. 340000.
 126500. 186500. 233230. 180000. 185000.  89500. 27

3875421174.8150687

さらに精度が下がったことを確認。

#### パターン５  
最終ステージをSVMに変更して実施。

In [63]:
# 引数

# L１のインスタンス
tree_stacking_0 = DecisionTreeRegressor()
tree_stacking_1 = DecisionTreeRegressor()
tree_stacking_2 = DecisionTreeRegressor()
tree_stacking_3 = DecisionTreeRegressor()
instance_a = [tree_stacking_0, tree_stacking_1, tree_stacking_2, tree_stacking_3]

neigh_stacking_0 = KNeighborsRegressor(n_neighbors=5)
neigh_stacking_1 = KNeighborsRegressor(n_neighbors=5)
neigh_stacking_2 = KNeighborsRegressor(n_neighbors=5)
neigh_stacking_3 = KNeighborsRegressor(n_neighbors=5)
instance_b = [neigh_stacking_0, neigh_stacking_1, neigh_stacking_2, neigh_stacking_3]

linear_stacking_0 = LinearRegression()
linear_stacking_1 = LinearRegression()
linear_stacking_2 = LinearRegression()
linear_stacking_3 = LinearRegression()
instance_c = [linear_stacking_0, linear_stacking_1, linear_stacking_2, linear_stacking_3]

model_instance = [instance_a, instance_b, instance_c]

# L2のインスタンス
l2__instance = SVR()



# インスタンス化
stacking_1 = ScratchStacking()
stacking_1.fit(X_train_scaled, y_train, model_instance, l2__instance)

# 推定
y_pred_b = stacking_1.predict(X_test_scaled)
stacking_1.MSE(y_test, y_pred_b)

[163373.45700179 163025.46140013 162952.13394242 163289.15229499
 162959.1673551  162989.19784696 163335.41859479 162950.94298557
 163208.0391862  163091.33815593 163299.52327678 163206.36772102
 163409.5385202  162942.59156801 162940.79794071 163032.20541667
 163403.98589579 163002.54352777 162994.48194581 163052.02616348
 163036.29383798 163018.00397263 162948.19039117 163341.32858044
 163309.74427842 162939.27448837 163371.05062854 162952.09458379
 163352.54140121 162952.54147634 163208.95361314 163360.6141231
 162949.5394835  163392.72956337 163322.22455342 163273.39239809
 163326.71181859 162946.87784952 163363.58978508 163327.60995369
 163385.30758111 163085.75698815 163280.42869413 163391.56153273
 163237.04412458 163155.76465104 162940.78912179 162956.94130143
 163263.80204678 162947.37622336 163244.45921027 162996.61974203
 163103.43927719 162974.61712767 163310.49183063 162941.83195583
 162972.24626586 163393.62873467 162981.98700318 162951.62351994
 162973.74326952 162959.87

7220395064.6483755

最さらに精度が下がったことを確認。

#### 考察  
ステージ０をSVM、最近傍法、決定木、最終ステージを線形回帰で構成したときにベースラインを上回ることができた。  
個別で精度が高いからと言って単独のモデルでスタッキングを構成しても精度は上がらず、 
むしろ下がる結果になった。  
また、いろいろなモデルを組み合わせるとしても、ステージ０と最終ステージの組み合わせによって  
かなり差が出ることが分かった。  

以上